In [1]:
# Import libraries
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import numpy as np
import os.path as op
from glob import glob
from nilearn.image import load_img, resample_to_img
import nilearn.plotting
from nilearn.maskers import NiftiLabelsMasker
from nilearn.datasets import fetch_atlas_aal
from ants import image_read, apply_transforms

# Fetch AAL atlas from nilearn
AAL = fetch_atlas_aal()

# Define important paths
bids = "/om2/user/mabdel03/files/Ravi_ISO_MRI/reformatted/"
derivatives = op.join(bids, "derivatives")
#qsiprep_dir = op.join(derivatives, "qsiprep_0.19.1")
#qsirecon_dir = op.join(derivatives, "qsirecon_0.19.1")
qsiprep_dir = op.join(derivatives, "qsiprep_0.20.0")
qsirecon_dir = op.join(derivatives, "qsirecon_0.20.0")
freesurfer_dir = op.join(derivatives, "freesurfer_7.4.1")
fmriprep_dir = op.join(derivatives, "fmriprep_23.2.0a3")
xcp_d_dir = op.join(derivatives, "xcp_d_0.6.1")

# Load phenotypic data, remove unnecessary columns
phenotype_csv_path = '/om2/user/mabdel03/files/Ravi_ISO_MRI/Metadata/patient_metadata_3t_raw.csv'
phenotype = pd.read_csv(phenotype_csv_path)
phenotype['subject'] = ["sub-"+subid.split('_')[0] for subid in phenotype['Patient_Year']]
phenotype = phenotype.set_index("subject")
phenotype = phenotype.sort_index()
phenotype = phenotype.drop(columns=['Unnamed: 0', 'Study', 'Date', 'Patient_Year', 'projid', 'study'])

# fMRI quality control parameters
fmri_qc_cutoff = 0.2 # If more than this proportion of data are motion outliers, do not pass QC

# Define important metrics
dwi_wholebrain_metrics = ["ad", "dti_fa", "md", "qa", "rd"]
dwi_csv_metrics = dwi_wholebrain_metrics + ['mean_length_mm', 'span_mm', 'curl', 'elongation',
       'diameter_mm', 'volume_mm3', 'trunk_volume_mm3', 'branch_volume_mm3',
       'total_surface_area_mm2']

# Get lists of subjects
subs_with_xcpd = [name.split("/")[-2] for name in glob(op.join(xcp_d_dir, "sub-*/"))]
subs_with_xcpd.sort()
subs_with_qsirecon = [name.split("/")[-2] for name in glob(op.join(qsirecon_dir, "sub-*/"))]
subs_with_qsirecon.sort()
all_subs = list(set(subs_with_xcpd) | set(subs_with_qsirecon))
all_subs.sort()
print(f"Number of subjects with XCP_D: {len(subs_with_xcpd)}")
print(f"Number of subjects with qsirecon: {len(subs_with_qsirecon)}")
print(f"Overlap in XCP_D and qsirecon: {len(set(subs_with_xcpd) & set(subs_with_qsirecon))}")
print(f"Number of total subjects: {len(all_subs)}")

Number of subjects with XCP_D: 555
Number of subjects with qsirecon: 563
Overlap in XCP_D and qsirecon: 552
Number of total subjects: 566


In [2]:
# Initialize dataframe, index columns as subject name
df = pd.DataFrame()
df["subject"] = all_subs
df = df.set_index("subject")
# Initialize everyone passing QC (update in loop if fails)
df['fmri_qc'] = np.ones(len(df))
df['dwi_qc'] = np.ones(len(df))
df['anat_qc'] = np.ones(len(df))
df['has_pheno'] = np.ones(len(df))

for sub in df.index:
    #sub = "sub-00807897"
    print(sub)
    
    ## Phenotype
    try:
        sub_row = phenotype.loc[sub]
        # Deal with multiple rows
        if np.size(sub_row['fu_year']) > 1:
            session_in_bids = int(glob(op.join(bids,sub,"ses-*"))[0].split("/")[-1][4:])
            sub_row = sub_row[sub_row['fu_year'] == session_in_bids]
            for key in sub_row.keys():
                df.loc[sub,'pheno_'+key] = sub_row[key].values[0]
        else:
            for key in sub_row.keys():
                df.loc[sub,"pheno_"+key] = sub_row[key]
    except Exception as e:
        print('PHENO ERROR:', e)
        df.loc[sub, 'has_phenp'] = 0
    
    ## Anat
    try:
        # Wholebrain measures
        brainmeasures_tsv_path = op.join(freesurfer_dir,sub,f"{sub}_brainmeasures.tsv")
        brainmeasures = pd.read_csv(brainmeasures_tsv_path, sep="\t")
        brainmeasures = brainmeasures.set_index("subject_id")
        brainmeasures = brainmeasures.drop(columns=["session_id"])
        for col in brainmeasures.columns:
            df.loc[sub, f"brainmeasures_{col}"] = brainmeasures[col].values[0]
        # Regionsurfacestats
        regionsurfacestats_tsv_path = op.join(freesurfer_dir,sub,f"{sub}_regionsurfacestats.tsv")
        regionsurfacestats = pd.read_csv(regionsurfacestats_tsv_path, sep="\t")
        # restrict to just rows where the column value of atlas is "AAL"
        atlas = "AAL"
        regionsurfacestats = regionsurfacestats[regionsurfacestats.atlas == atlas]
        # Set index to be {structname}_{hemisphere}
        regionsurfacestats["StructName"] = regionsurfacestats["hemisphere"] + "_" + regionsurfacestats["StructName"]
        regionsurfacestats = regionsurfacestats.set_index("StructName")
        # remove columns we don't need
        regionsurfacestats = regionsurfacestats.drop(columns=["subject_id", "session_id", "atlas", "hemisphere"])
        for structname in regionsurfacestats.index:
            for col in regionsurfacestats.columns:
                df.loc[sub, f"regionsurfacestats_{atlas}_{structname}_{col}"] = regionsurfacestats.loc[structname, col]

    except Exception as e:
        print('ANAT ERROR:', e)
        df.loc[sub, 'anat_qc'] = 0
    
    ## DWI
    try:
        ## EVENTUALLY PUT DWI QC HERE

        # Load tissue segmentation
        dwi_dseg_path = op.join(qsiprep_dir,sub,"anat",f"{sub}_dseg.nii.gz")
        dwi_dseg = load_img(dwi_dseg_path)
        # Load DWI-resolution file for resampling
        dwi_mask_path = glob(op.join(qsiprep_dir,sub,"ses*","dwi","*_desc-brain_mask.nii.gz"))[0]
        dwi_mask = load_img(dwi_mask_path)
        # Resample dseg to DWI mask
        dwi_dseg_resampled = resample_to_img(dwi_dseg, dwi_mask, interpolation="nearest")
        # Extract just white matter
        dwi_white_matter_indices = (dwi_dseg_resampled.get_fdata() == 3)

        # Warp AAL atlas to DWI space
        MNI_to_DWI_xfm_path = glob(op.join(qsiprep_dir,sub,"anat","*from-MNI152NLin2009cAsym_to-T1w_mode-image_xfm.h5"))[0]
        AAL_in_DWI_space = apply_transforms(fixed=image_read(dwi_mask_path), moving=image_read(AAL.maps),
                                            transformlist=MNI_to_DWI_xfm_path, interpolator='nearestNeighbor')
        # save it out, then load in nilearn so we can use later
        AAL_in_DWI_space.to_filename(op.join(qsiprep_dir, sub, 'anat', f"{sub}_space-T1w_desc-AAL.nii.gz"))
        AAL_in_DWI_space = load_img(op.join(qsiprep_dir, sub, 'anat', f"{sub}_space-T1w_desc-AAL.nii.gz"))

        ## Get wholebrain average DWI metrics
        for metric in dwi_wholebrain_metrics:
            # Load whole brain metric
            dwi_metric_path = glob(op.join(qsirecon_dir, sub, 'ses-*','dwi', f'*{metric}_gqiscalar.nii.gz'))[0]
            dwi_metric = load_img(dwi_metric_path)
            df.loc[sub, f"dwi_white_matter_average_{metric}"] = dwi_metric.get_fdata()[dwi_white_matter_indices].mean()
            # Now parcellate it with AAL
            AAL_masker_dwi_space = NiftiLabelsMasker(labels_img=AAL_in_DWI_space, mask_img=dwi_mask, labels=AAL.labels)
            dwi_metric_parcellated = AAL_masker_dwi_space.fit_transform(dwi_metric)[0,:]
            for dwi_value, label in zip(dwi_metric_parcellated, AAL.labels):
                df.loc[sub, f"dwi_AAL_{label}_{metric}"] = dwi_value

        ## Extract tract-wise metrics of interest
        # Load DSI Studio Autotrack CSV
        autotrack_stats_csv_path = glob(op.join(qsirecon_dir, sub, 'ses-*','dwi', '*desc-preproc_AutoTrackGQI.csv'))[0]
        autotrack_stats = pd.read_csv(autotrack_stats_csv_path)
        autotrack_stats = autotrack_stats.set_index("bundle_name")
        for tract in autotrack_stats.index:
            for metric in dwi_csv_metrics:
                df.loc[sub, f"dwi_{tract}_{metric}"] = autotrack_stats.loc[tract, metric]
                
    except Exception as e:
        print('DWI ERROR:', e)
        df.loc[sub, 'dwi_qc'] = 0
        
    ## fMRI    
    try:        
        # fMRI QC Check
        outlier_tsv_path = glob(op.join(xcp_d_dir, sub, 'ses-*', 'func', '*outliers.tsv'))[0]
        outlier_tsv = pd.read_csv(outlier_tsv_path, sep="\t")
        n_outliers = sum(outlier_tsv['framewise_displacement'])
        n_volumes = len(outlier_tsv)
        proportion_outliers = n_outliers / n_volumes
        if proportion_outliers > fmri_qc_cutoff:
            df.loc[sub, "fmri_qc"] = 0

        # Load brain masks
        fmri_mask_path = glob(op.join(fmriprep_dir, sub, 'ses-*', 'func', '*space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz'))[0]
        fmri_mask = load_img(fmri_mask_path)
        fmri_mask_inds = (fmri_mask.get_fdata() == 1)
        fmri_anat_mask_path = glob(op.join(fmriprep_dir, sub, 'ses-*', 'anat', '*space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz'))[0]
        fmri_anat_mask = load_img(fmri_anat_mask_path)
        fmri_anat_mask_inds = (fmri_anat_mask.get_fdata() == 1)

        # Load ALFF
        wholebrain_alff_path = glob(op.join(xcp_d_dir, sub, 'ses-*', 'func', f'*space-MNI152NLin2009cAsym_alff.nii.gz'))[0]
        wholebrain_alff = load_img(wholebrain_alff_path)
        # Calculate average ALFF in brain
        wholebrain_average_alff = wholebrain_alff.get_fdata()[fmri_mask_inds].mean()
        df.loc[sub, "bold_wholebrain_average_alff"] = wholebrain_average_alff
        # Mask ALFF by AAL atlas
        AAL_masker = NiftiLabelsMasker(labels_img=AAL.maps, labels=AAL.labels, mask_img=fmri_mask)
        alff_parcellated = AAL_masker.fit_transform(wholebrain_alff)[0,:]
        for alff_value, label in zip(alff_parcellated, AAL.labels):
            df.loc[sub, f"bold_AAL_{label}_alff"] = alff_value

        # Load T1T2 Ratio
        wholebrain_t1t2_path = glob(op.join(xcp_d_dir, sub, 'ses-*', 'anat', f'*MNI152NLin2009cAsym_desc-preproc_T1wT2wRatio.nii.gz'))[0]
        wholebrain_t1t2 = load_img(wholebrain_t1t2_path)
        # Calculate average T1T2 Ratio in brain
        wholebrain_average_t1t2 = wholebrain_t1t2.get_fdata()[fmri_anat_mask_inds].mean()
        df.loc[sub, "bold_wholebrain_average_t1t2ratio"] = wholebrain_average_t1t2
        # Mask T1T2 Ratio by AAL atlas
        t1t2_parcellated = AAL_masker.fit_transform(wholebrain_t1t2)[0,:]
        for t1t2_value, label in zip(t1t2_parcellated, AAL.labels):
            df.loc[sub, f"bold_AAL_{label}_t1t2ratio"] = t1t2_value
            
    except Exception as e:
        print('BOLD ERROR:', )
        df.loc[sub, "fmri_qc"] = 0
        
df.to_csv(op.join(bids,"code","slm_data3.csv"))

sub-00582981
sub-00807897
sub-01172523
sub-01841461
sub-01878130
sub-02158395
sub-02371278
sub-02695917
sub-02732911
sub-02950975
sub-03052480
sub-03060782
sub-03070130
sub-03227207
sub-03340627
sub-03902977
sub-03918123
sub-04319814
sub-04981936
sub-04984547
sub-05001777
sub-05225665
sub-05389430
sub-05427601
sub-05859529
sub-06042154
sub-06131380
sub-06152191
sub-06630618
sub-06673533
sub-06702361
sub-06764103
sub-07053099
sub-07117738
sub-07195200
sub-08072948
sub-08098966
sub-08245033
sub-08464714
sub-08481297
sub-08558869
sub-08587260
sub-08899955
sub-09108223
sub-09508427
sub-09790439
sub-09892462
sub-10042633
sub-10262879
sub-10371937
DWI ERROR: File not found: '/om2/user/mabdel03/files/Ravi_ISO_MRI/reformatted/derivatives/qsiprep_0.20.0/sub-10371937/anat/sub-10371937_dseg.nii.gz'
sub-10516212
sub-10665684
sub-11486113
sub-11626867
sub-12317427
sub-12357443
sub-12766954
sub-13026999
sub-13103757
sub-13269830
sub-13282409


/om2/user/smeisler/anaconda3/envs/nipype/lib/python3.8/site-packages/scipy/ndimage/measurements.py:684: RuntimeWarning:

invalid value encountered in true_divide



sub-13346850
sub-13360984
sub-13380780
sub-13432899
sub-13633244
sub-13747771


/om2/user/smeisler/anaconda3/envs/nipype/lib/python3.8/site-packages/scipy/ndimage/measurements.py:684: RuntimeWarning:

invalid value encountered in true_divide

/om2/user/smeisler/anaconda3/envs/nipype/lib/python3.8/site-packages/scipy/ndimage/measurements.py:684: RuntimeWarning:

invalid value encountered in true_divide



sub-13885104
sub-13942030
sub-14050539


...

sub-80487613
sub-80573076
sub-80686912
sub-80930779
sub-81185560
BOLD ERROR:
sub-81404254
sub-81622344
sub-81639479
sub-81810992
sub-82003527
sub-82273425
sub-82320165
sub-82325110
sub-82335720
sub-82366015
sub-82520217
sub-82728899
sub-82736939
sub-82780217
sub-82890542
sub-83092060
sub-83112293
sub-83216408


/om2/user/smeisler/anaconda3/envs/nipype/lib/python3.8/site-packages/scipy/ndimage/measurements.py:684: RuntimeWarning:

invalid value encountered in true_divide

/om2/user/smeisler/anaconda3/envs/nipype/lib/python3.8/site-packages/scipy/ndimage/measurements.py:684: RuntimeWarning:

invalid value encountered in true_divide



sub-83887227
sub-83944153
sub-84248280
sub-84537270
sub-84896566
sub-84916951
sub-85065931
sub-85508351
sub-85520889
BOLD ERROR:
sub-85527016
sub-85539934
sub-86880662
sub-86901774
sub-86910081
sub-86934089
sub-87386573
sub-87585034
sub-87713446
sub-87821751
sub-87942151
sub-88029789
sub-88049747
sub-88153300
sub-88182963
sub-88488240
sub-88799920
sub-88910757
sub-88961000
sub-89292806
sub-89515666
sub-89835239
sub-89862872
sub-89912411
sub-90076229
sub-90172292
sub-90173045
sub-90224563
sub-90254005
sub-90439144
sub-90544000
sub-90805280
sub-90821208
sub-90977195
sub-91231332
sub-91349809
sub-91949579
sub-92276597
sub-93041854
sub-93138484
sub-93417026
sub-93435488


/om2/user/smeisler/anaconda3/envs/nipype/lib/python3.8/site-packages/scipy/ndimage/measurements.py:684: RuntimeWarning:

invalid value encountered in true_divide

/om2/user/smeisler/anaconda3/envs/nipype/lib/python3.8/site-packages/scipy/ndimage/measurements.py:684: RuntimeWarning:

invalid value encountered in true_divide



sub-93493992
sub-93667416
sub-93920166
sub-93966278
sub-94225596
sub-94549523
sub-94630355
sub-94803738
sub-94937061
sub-95065654
sub-95252909
sub-95281012
sub-95373147
sub-95439194
sub-95620227
sub-95638231
sub-95902182
sub-96177193
sub-96314498
sub-96351191
sub-96370720
sub-96915597
sub-97478169
sub-97498839
sub-97775875
sub-97778512
sub-97989289
sub-98273670
sub-98844813
sub-98919041
sub-99147061
sub-99151323
sub-99516006
sub-99693576
sub-99893592
sub-99911705
BOLD ERROR:


In [91]:
df

,fmri_qc,dwi_qc,anat_qc,has_pheno,pheno_fu_year,pheno_age_at_visit,pheno_cogn_global,pheno_cesdsum,pheno_r_depres,pheno_bmi,...,bold_AAL_Cerebelum_10_L_t1t2ratio,bold_AAL_Cerebelum_10_R_t1t2ratio,bold_AAL_Vermis_1_2_t1t2ratio,bold_AAL_Vermis_3_t1t2ratio,bold_AAL_Vermis_4_5_t1t2ratio,bold_AAL_Vermis_6_t1t2ratio,bold_AAL_Vermis_7_t1t2ratio,bold_AAL_Vermis_8_t1t2ratio,bold_AAL_Vermis_9_t1t2ratio,bold_AAL_Vermis_10_t1t2ratio
subject,,,,,,,,,,,,,,,,,,,,,
sub-00582981,1.0,1.0,1.0,1.0,0.0,76.104038,0.448405,0.0,4.0,28.585375,...,2.28769,2.34018,2.480218,2.588595,2.961879,3.373826,3.499202,2.792633,2.77761,1.567428
sub-00807897,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub-01172523,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub-01841461,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub-01878130,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sub-99151323,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub-99516006,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub-99693576,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
